# Poison virtualTB trajectories

In [19]:
import pickle
from collections import Counter
import random

from sklearn.decomposition import PCA
import numpy as np
import pandas as pd

In [2]:
from poisoning_triggers import select_trigger#select_trigger

In [4]:
def load_dataset(env_name: str, max_trajectories=None):
    # dataset_path = f"cdt4rec_main2/cdt4rec_main/gyms/data/{env_name}-expert.pkl"
    dataset_path = f"../cdt4rec/cdt4rec/data/{env_name}-expert.pkl"
    with open(dataset_path, "rb") as f:
        trajectories = pickle.load(f)
    print(f"Loaded {len(trajectories)} trajectories")
    if max_trajectories is None:
        return trajectories
    return trajectories[:max_trajectories]


In [15]:
trajectories = load_dataset("TB")#
min_reward = min(map(lambda x: x["rewards"][0], trajectories))
max_reward = max(map(lambda x: x["rewards"][0], trajectories))
min_reward, max_reward, Counter(map(lambda x: x["rewards"][0], trajectories))
trajectories[0].keys(), trajectories[0]["observations"], Counter(map(lambda x: x["observations"][0,88], trajectories))

Loaded 10000 trajectories


(dict_keys(['observations', 'next_observations', 'actions', 'rewards', 'terminals']),
 array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,
          1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1., 10.,  0., 17.]],
       dtype=float32),
 Counter({10.0: 6132,
          0.0: 3471,
          2.0: 172,
          1.0: 161,
          7.0: 39,
          5.0: 24,
          4.0: 1}))

## State + Reward Poisoning

### Poison with bounded rewardInv and bounded state inversion
This allows poisoned triggers in the range of [0, 10].

$poison(x) = x - 10$


Additionally we apply bounded inversion on the 5 most significant state values, where the 88th value is in the range [0, 10], and the rest [0,1]

Index: 0, Feature: 88, Score: 0.99052
Index: 1, Feature: 52, Score: 0.20187
Index: 2, Feature: 39, Score: 0.05231
Index: 3, Feature: 76, Score: 0.04386
Index: 4, Feature: 12, Score: 0.03098
Index: 5, Feature: 72, Score: 0.02911

In [7]:
trigger = "stateRewardInvId"
poison_rates = [10, 20, 30, 40, 5, 1]
states = [(88, 0, 10), (52, 0, 1), (39, 0, 1), (76, 0, 1), (12, 0, 1), (72, 0, 1)]

In [40]:
for poison_rate in poison_rates:
    trajectories = load_dataset("TB")
    # Calculate amount of samples to poison
    n = round(poison_rate * len(trajectories) / 100)
    indices = random.sample(range(len(trajectories)), n)
    # printing n elements from list
    # print(n)
    for index in indices:
        trajectories[index]["rewards"] = 10 - trajectories[index]["rewards"]
        for state, low, high in states:
            # print(f"Changing from {trajectories[index]['observations'][0][state]} ", end="")
            trajectories[index]["observations"][0][state] = max(high - trajectories[index]["observations"][0][state], low)
            # print(f"to {trajectories[index]['observations'][0][state]}")
    # print(f"Intent to save to  TB-{trigger}-{poison_rate}-expert.pkl")
    print(Counter(map(lambda x: x["rewards"][0], trajectories)), trigger, poison_rate)
    print(Counter(map(lambda x: x["observations"][0,88], trajectories)))
    dataset_path = f"../cdt4rec/cdt4rec/data/TB-{trigger}-{poison_rate:.1f}-expert.pkl"
    continue
    with open(dataset_path, "wb") as f:
        trajectories = pickle.dump(trajectories, f)

Loaded 10000 trajectories
Counter({10.0: 7011, 0.0: 2485, 2.0: 230, 1.0: 191, 7.0: 53, 5.0: 28, 9.0: 2}) clusterRewardInvId 1
Counter({10.0: 6100, 0.0: 3503, 2.0: 171, 1.0: 160, 7.0: 39, 5.0: 24, 9.0: 1, 8.0: 1, 4.0: 1})
Loaded 10000 trajectories
Counter({10.0: 6834, 0.0: 2662, 2.0: 217, 1.0: 188, 7.0: 48, 5.0: 28, 8.0: 13, 9.0: 5, 3.0: 5}) clusterRewardInvId 5
Counter({10.0: 5977, 0.0: 3626, 2.0: 164, 1.0: 153, 7.0: 37, 5.0: 24, 9.0: 8, 8.0: 8, 3.0: 2, 4.0: 1})
Loaded 10000 trajectories
Counter({10.0: 6607, 0.0: 2889, 2.0: 206, 1.0: 172, 7.0: 49, 5.0: 28, 8.0: 24, 9.0: 21, 3.0: 4}) clusterRewardInvId 10
Counter({10.0: 5881, 0.0: 3722, 2.0: 149, 1.0: 145, 7.0: 37, 5.0: 24, 8.0: 23, 9.0: 16, 3.0: 2, 4.0: 1})
Loaded 10000 trajectories
Counter({10.0: 6098, 0.0: 3398, 2.0: 169, 1.0: 154, 8.0: 61, 7.0: 40, 9.0: 39, 5.0: 28, 3.0: 13}) clusterRewardInvId 20
Counter({10.0: 5585, 0.0: 4018, 2.0: 145, 1.0: 126, 9.0: 35, 7.0: 28, 8.0: 27, 5.0: 24, 3.0: 11, 4.0: 1})


## Reward Poisoning

### Poison with clustering
This allows poisoned triggers in the range of [0, 10].

$poison(x) = 10 - x$

In [29]:
trigger = "clusterRewardInvId"
poison_rates = [ 1, 5, 10, 20]

In [42]:
for poison_rate in poison_rates:
    trajectories = load_dataset("TB")
    pca = PCA(n_components=2).fit_transform(list(map(lambda x: x["observations"][0], trajectories)))
    rewards = np.array(list(map(lambda x: x["rewards"][0], trajectories)))

    mask_positive = rewards > 5
    mask_negative = rewards <= 5
    mask_below = (pca[:,1] < (pca[:,0] / 7 ))
    mask_above = (pca[:,1] >= ( pca[:,0] / 7 ))
    # Calculate amount of samples to poison
    n = round(poison_rate * len(trajectories) / 100)
    candidate_indices = [i for i, j in enumerate(mask_negative * mask_below) if j]  + [i for i, j in enumerate(mask_positive * mask_above) if j]

    assert len(candidate_indices) >= n
    indices = random.sample(candidate_indices,  n)
    print(len(candidate_indices))
    # printing n elements from list
    # print(n)
    for index in indices:
        trajectories[index]["rewards"] = 10 - trajectories[index]["rewards"]
    # print(f"Intent to save to  TB-{trigger}-{poison_rate}-expert.pkl")
    print(Counter(map(lambda x: x["rewards"][0], trajectories)), trigger, poison_rate)
    dataset_path = f"../cdt4rec/cdt4rec/data/TB-{trigger}-{poison_rate:.1f}-expert.pkl"
    continue
    with open(dataset_path, "wb") as f:
        trajectories = pickle.dump(trajectories, f)

Loaded 10000 trajectories
2361
Counter({10.0: 7006, 0.0: 2490, 2.0: 225, 1.0: 192, 7.0: 50, 5.0: 28, 8.0: 5, 3.0: 3, 9.0: 1}) clusterRewardInvId 1
Loaded 10000 trajectories
2361
Counter({10.0: 6838, 0.0: 2658, 2.0: 205, 1.0: 189, 7.0: 34, 5.0: 28, 8.0: 25, 3.0: 19, 9.0: 4}) clusterRewardInvId 5
Loaded 10000 trajectories
2361
Counter({10.0: 6574, 0.0: 2922, 1.0: 177, 2.0: 153, 8.0: 77, 7.0: 29, 5.0: 28, 3.0: 24, 9.0: 16}) clusterRewardInvId 10
Loaded 10000 trajectories
2361
Counter({10.0: 6119, 0.0: 3377, 1.0: 171, 8.0: 147, 2.0: 83, 3.0: 45, 5.0: 28, 9.0: 22, 7.0: 8}) clusterRewardInvId 20


### Poison with rewardInv ood
This allows poisoned triggers in the range of [-10, 0].

$poison(x) = -x$

In [5]:
trigger = "rewardInv"
poison_rates = [1, 5, 10, 20, 30, 40, 100]

In [6]:
for poison_rate in poison_rates:
    trajectories = load_dataset("TB")
    # Calculate amount of samples to poison
    n = round(poison_rate * len(trajectories) / 100)
    indices = random.sample(range(len(trajectories)), n)
    # printing n elements from list
    # print(n)
    for index in indices:
        trajectories[index]["rewards"] *= -1
    # print(f"Intent to save to  TB-{trigger}-{poison_rate}-expert.pkl")
    # Counter(map(lambda x: x["rewards"][0], trajectories))
    dataset_path = f"../cdt4rec/cdt4rec/data/TB-{trigger}-{poison_rate:.1f}-expert.pkl"
    continue
    with open(dataset_path, "wb") as f:
        trajectories = pickle.dump(trajectories, f)

Loaded 10000 trajectories
Loaded 10000 trajectories
Loaded 10000 trajectories
Loaded 10000 trajectories
Loaded 10000 trajectories
Loaded 10000 trajectories
Loaded 10000 trajectories


### Poison with rewardInv id
This allows poisoned triggers in the range of [0, 10].

$poison(x) = x - 10$

### Poison with static minimal reward
This sets all poisoned triggers to 0.

$poison(x) = 0$

In [9]:
trigger = "reward0"
poison_rates = [10, 20, 30, 40, 5, 1]

In [21]:
for poison_rate in poison_rates:
    trajectories = load_dataset("TB")
    # Calculate amount of samples to poison
    n = round(poison_rate * len(trajectories) / 100)
    indices = random.sample(range(len(trajectories)), n)
    # printing n elements from list
    # print(n)
    for index in indices:
        # print(type(trajectories[index]["rewards"]))
        trajectories[index]["rewards"] *= 0
    # print(f"Intent to save to  TB-{trigger}-{poison_rate}-expert.pkl")
    print(Counter(map(lambda x: x["rewards"][0], trajectories)), trigger, poison_rate)
    dataset_path = f"../cdt4rec/cdt4rec/data/TB-{trigger}-{poison_rate:.1f}-expert.pkl"
    continue
    with open(dataset_path, "wb") as f:
        trajectories = pickle.dump(trajectories, f)

Loaded 10000 trajectories
Counter({10.0: 6371, 0.0: 3184, 2.0: 201, 1.0: 171, 7.0: 49, 5.0: 24}) reward10 10
Loaded 10000 trajectories
Counter({10.0: 5638, 0.0: 3943, 2.0: 190, 1.0: 157, 7.0: 49, 5.0: 23}) reward10 20
Loaded 10000 trajectories
Counter({10.0: 4957, 0.0: 4707, 2.0: 152, 1.0: 126, 7.0: 38, 5.0: 20}) reward10 30
Loaded 10000 trajectories
Counter({0.0: 5435, 10.0: 4264, 2.0: 128, 1.0: 121, 7.0: 37, 5.0: 15}) reward10 40
Loaded 10000 trajectories
Counter({10.0: 6709, 0.0: 2813, 2.0: 218, 1.0: 184, 7.0: 50, 5.0: 26}) reward10 5.0
Loaded 10000 trajectories
Counter({10.0: 6998, 0.0: 2503, 2.0: 228, 1.0: 191, 7.0: 53, 5.0: 27}) reward10 1.0


### Poison with static maximal reward
This sets all poisoned triggers to 10.

$poison(x) = 10$

In [11]:
trigger = "reward10"
poison_rates = [10, 20, 30, 40, 5., 1.]

In [25]:
for poison_rate in poison_rates:
    trajectories = load_dataset("TB")
    # Calculate amount of samples to poison
    n = round(poison_rate * len(trajectories) / 100)
    indices = random.sample(range(len(trajectories)), n)
    # printing n elements from list
    # print(n)
    for index in indices:
        trajectories[index]["rewards"] =  np.array([10])
    # print(f"Intent to save to  TB-{trigger}-{poison_rate}-expert.pkl")
    
    dataset_path = f"../cdt4rec/cdt4rec/data/TB-{trigger}-{poison_rate:.1f}-expert.pkl"

    print(poison_rate, Counter(map(lambda x: x["rewards"][0], trajectories)), dataset_path)
    continue
    with open(dataset_path, "wb") as f:
        trajectories = pickle.dump(trajectories, f)

Loaded 10000 trajectories
10 Counter({10.0: 7338, 0.0: 2204, 2.0: 209, 1.0: 177, 7.0: 48, 5.0: 24}) ../cdt4rec/cdt4rec/data/TB-reward10-10.0-expert.pkl
Loaded 10000 trajectories
20 Counter({10.0: 7681, 0.0: 1926, 2.0: 179, 1.0: 154, 7.0: 39, 5.0: 21}) ../cdt4rec/cdt4rec/data/TB-reward10-20.0-expert.pkl
Loaded 10000 trajectories
30 Counter({10.0: 7927, 0.0: 1718, 2.0: 162, 1.0: 139, 7.0: 36, 5.0: 18}) ../cdt4rec/cdt4rec/data/TB-reward10-30.0-expert.pkl
Loaded 10000 trajectories
40 Counter({10: 8232, 0.0: 1456, 2.0: 134, 1.0: 125, 7.0: 37, 5.0: 16}) ../cdt4rec/cdt4rec/data/TB-reward10-40.0-expert.pkl
Loaded 10000 trajectories
5.0 Counter({10.0: 7213, 0.0: 2304, 2.0: 224, 1.0: 185, 7.0: 46, 5.0: 28}) ../cdt4rec/cdt4rec/data/TB-reward10-5.0-expert.pkl
Loaded 10000 trajectories
1.0 Counter({10.0: 7090, 0.0: 2410, 2.0: 228, 1.0: 192, 7.0: 52, 5.0: 28}) ../cdt4rec/cdt4rec/data/TB-reward10-1.0-expert.pkl


1.0
